In [ ]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
# app.py
import streamlit as st
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import load_model

# --- CONFIGURATION ---

# Available models (filename: display name)
model_paths = {
    "Model A :LSTM": "emotion_recognition_model.h5",
    "Model B :EfficientNet": "model_b.h5",
    "Model C :ResNet50": "model_c.h5"
}

# Define class labels (must match your training)
class_names = np.array(["Angry", "Happy", "Sad", "Neutral", "Fear", "Surprise"])

# --- STREAMLIT UI ---

st.title("🎤 Emotion Detection from Audio")
st.write("Upload a WAV file and select a model to predict the emotion.")

# Model selector
selected_model_name = st.selectbox("Choose a model:", list(model_paths.keys()))
selected_model_path = model_paths[selected_model_name]

# Load the selected model
@st.cache_resource
def load_selected_model(path):
    return load_model(path)

model = load_selected_model(selected_model_path)

# File uploader
uploaded_file = st.file_uploader("Upload a WAV file", type=["wav"])

if uploaded_file is not None:
    # Load audio
    y, sr = librosa.load(uploaded_file, sr=22050)

    # Audio player
    st.audio(uploaded_file, format='audio/wav')

    # Mel Spectrogram
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_db = librosa.power_to_db(mel, ref=np.max)

    # Show spectrogram
    fig, ax = plt.subplots(figsize=(6, 4))
    img = librosa.display.specshow(mel_db, sr=sr, x_axis='time', y_axis='mel', ax=ax)
    ax.set(title='Mel Spectrogram')
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    st.pyplot(fig)

    # --- Preprocessing based on model input shape ---
    model_input_shape = model.input_shape  # e.g. (None, 128, 128, 1) or (None, 128, 1) or (None, 128)

    mel_resized = cv2.resize(mel_db, (128, 128))
    mel_normalized = (mel_resized - mel_resized.min()) / (mel_resized.max() - mel_resized.min())

    if len(model_input_shape) == 4:
        # Likely Conv2D: (batch, 128, 128, 1)
        input_array = mel_normalized.reshape(1, 128, 128, 1)

    elif len(model_input_shape) == 3:
        # Likely Conv1D or LSTM: (batch, 128, 1) or (batch, 128)
        # Reduce to 1D over frequency
        mean_vector = np.mean(mel_normalized, axis=0)
        if model_input_shape[-1] == 1:
            input_array = mean_vector.reshape(1, 128, 1)
        else:
            input_array = mean_vector.reshape(1, 128)

    else:
        st.error(f"Unsupported model input shape: {model_input_shape}")
        st.stop()

    # Predict
    predictions = model.predict(input_array)[0]
    predicted_index = np.argmax(predictions)
    predicted_label = class_names[predicted_index]

    # Results
    st.success(f"**Predicted Emotion:** {predicted_label} ({predictions[predicted_index]*100:.2f}%)")

    st.subheader("Class Probabilities")
    for i, prob in enumerate(predictions):
        st.write(f"{class_names[i]}: {prob*100:.2f}%")


Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Add your auth token
ngrok.set_auth_token("2x6ZVT1pcHmfFjHJL40WoGxnn40_2PjYgzDptE7s1h7ypUyeo")

# Kill previous instances (just in case)
!pkill streamlit

# Run Streamlit app
!streamlit run app.py &> streamlit_log.txt &

# Open the port using ngrok
public_url = ngrok.connect(addr=8501)
print(f"🌐 Streamlit app is live at: {public_url}")


🌐 Streamlit app is live at: NgrokTunnel: "https://5b06-34-23-144-134.ngrok-free.app" -> "http://localhost:8501"
